In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from MongoCRUD import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "changeme"

# Connect to database via CRUD Module
db = AnimalShelter()

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'], inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    
        html.Center(html.A(
        href='https://snhu.edu',
        target='_blank',  # Open link in a new tab
        children=[
            html.Img(
                src='data:image/png;base64,{}'.format(encoded_image.decode()),
                style={'width': '10%'}
            )
        ]
    )),
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Bryan Pirrone - CS-340'))),
    dcc.Dropdown(['Reset', 
                  'Water Rescue', 
                  'Mountain or Wilderness Rescue',
                  'Disaster or Individual Tracking'], 'Select Rescue Type', id='dropdown'),
    html.Hr(),
    html.Div(
        

    ),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         page_size=10,
                         sort_action='native',
                         row_selectable = "single",

                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id', 'data'),
              [Input('dropdown', 'value')])
def update_dashboard(value):
    # Define the initial DataFrame and columns
    df = pd.DataFrame.from_records(db.read({}))
    df.drop(columns=['_id'], inplace=True)
    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]

    if value == 'Reset':
        data = df.to_dict('records')
    elif value == 'Water Rescue':
        breeds = ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]
        filtered_df = df[df['breed'].isin(breeds)]
        data = filtered_df.to_dict('records')
    elif value == 'Mountain or Wilderness Rescue':
        breeds = ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]
        filtered_df = df[df['breed'].isin(breeds)]
        data = filtered_df.to_dict('records')
    elif value == 'Disaster or Individual Tracking':
        breeds = ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]
        filtered_df = df[df['breed'].isin(breeds)]
        data = filtered_df.to_dict('records')
    else:
        data = df.to_dict('records')

    return data
# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "data")])
def update_graphs(viewData):
    

    return [
        dcc.Graph(            
            figure = px.pie(df, names='breed', title='Preferred Animals')
        )    
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if not selected_columns:
        return []
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows"),
     Input('datatable-id', 'style_data_conditional')])
def update_map(derived_virtual_data, index, style_data_conditional):
    if not derived_virtual_data:  # Check if data is empty
        raise PreventUpdate  

    dff = pd.DataFrame.from_dict(derived_virtual_data)

 # Because we only allow single row selection, the list can 
 # be converted to a row index here
    if not index or len(index) == 0:  # Handle case when no row is selected
        row = 0
    else: 
        row = index[0]

# Austin TX is at [30.75,-97.48]
    return [
    dl.Map(style={'width': '1000px', 'height': '500px'},
       center=[30.75,-97.48], zoom=10, children=[
       dl.TileLayer(id="base-layer-id"),
       # Marker with tool tip and popup
       # Column 13 and 14 define the grid-coordinates for 
       # the map
       # Column 4 defines the breed for the animal
       # Column 9 defines the name of the animal
       dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]],
          children=[
          dl.Tooltip(dff.iloc[row,4]),
          dl.Popup([
             html.H1("Animal Name:"),
            html.H1(dff.iloc[row,9])
         ])
      ])
   ])
]
    

app.run_server(debug=True)


Dash app running on http://127.0.0.1:29988/
